In [1]:
import os
os.chdir('..')
os.environ['LINE_PROFILE'] = '1'

In [2]:
from datetime import timezone
from pathlib import Path

import polars as pl
from dateutil import parser

from config import DataFrequency, BINANCE_DATA_DIR, TradeType

from util.ts_manager import TSManager
from aws.kline.parse import read_csv
from aws.client_async import AwsKlineClient

In [18]:
symbol = 'XRPUSDT'
time_interval = '1m'
base_dir = AwsKlineClient.get_base_dir(TradeType.um_futures, data_freq=DataFrequency.daily)
aws_dir = BINANCE_DATA_DIR / 'aws_data' / base_dir / symbol / time_interval
aws_kline_files = sorted(aws_dir.glob(f'*.zip'))
df = pl.concat([read_csv(f) for f in aws_kline_files])

In [ ]:
import importlib
import util
import util.ts_manager
importlib.reload(util.ts_manager)

# 提取月份并添加到 DataFrame 中
df = df.with_columns(
    pl.col("candle_begin_time").dt.strftime("%Y-%m").alias("month")
)

# 按月份拆分 DataFrame
split_dfs = df.partition_by("month")

data_dir = BINANCE_DATA_DIR / 'parsed_data' / TradeType.um_futures.value / 'klines' / symbol / time_interval

mgr = TSManager(data_dir, data_freq=DataFrequency.monthly)

for df1 in split_dfs:
    df1 = df1.drop('month')
    mgr.update(df1)

In [ ]:
pl.read_parquet('/home/lostleaf/crypto_data/binance_data/parsed_data/um_futures/klines/XRPUSDT/1m/202401.pqt')